In [2]:
import numpy as np
import pandas as pd
from flask import Flask, jsonify, request
import pandas as pd
from simpletransformers.classification import ClassificationModel, ClassificationArgs
from flask_cors import CORS
import json

LABELS = ['Trivial','Minor','Medium', 'Major', 'Huge','Enormous','Epic']
S_LABELS = [1,2,3,5,8,13,21]

app = Flask(__name__)
CORS(app)

df = pd.read_csv('C:\\Users\\jugas\\PythonWS\\jira-data.csv')
df.index=df.issuekey
df=df.drop(['Unnamed: 0', 'td'], axis=1)

df['complexity'] = pd.cut(df['storypoint'], bins=[0.5,1.5,2.5,3.5,5.5,8.5,13.5,20.5], labels=LABELS)

model = ClassificationModel('roberta',  'models/tf_roberta-base_20210818-144353', use_cuda=False)

@app.route('/summary')
def get_summary():
    return jsonify(summary=[{'complexity' : k, 'count': v}for k,v in df['complexity'].value_counts().iteritems()]), 200


@app.route('/projects')
def get_projects():
    return jsonify(projects=['all', *[p for p in df['project'].unique()]]), 200

@app.route('/jiras')
def get_jiras():
    num_results = int(request.args['num_results'])
    project = request.args['project']
    odf = df.copy()
    if project != 'all':
        odf = odf.loc[odf['project'] == project]
    return odf[:num_results].to_json(orient='records'), 200

@app.route('/predictions', methods=['POST'])    
def predictions():
    data = json.loads(request.data)
    text = data['text']
    prediction =  model.predict([text])
    
    return {
        'text' : text,
        'prediction' : S_LABELS[prediction[0][0]]
    }, 200


@app.route('/planning', methods=['POST'])    
def planning():
    data = json.loads(request.data)
    jiras = data['jiras']

    odf=df.loc[df['issuekey'].isin(jiras)]
    text = (odf['title'] + odf['description']).to_list()

    predictions, _ =  model.predict(text)
    
    return {
        'predictions' : [S_LABELS[prediction] for prediction in predictions]
    }, 200

if __name__ == '__main__':   
    app.run(host='localhost', port=5000, debug=False, threaded=True)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://localhost:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [02/Sep/2021 12:51:46] "GET /jiras?num_results=15&project=all HTTP/1.1" 200 -
127.0.0.1 - - [02/Sep/2021 12:51:54] "OPTIONS /planning HTTP/1.1" 200 -
100%|██████████| 1/1 [00:00<00:00,  1.42it/s]
127.0.0.1 - - [02/Sep/2021 12:52:02] "POST /planning HTTP/1.1" 200 -
127.0.0.1 - - [02/Sep/2021 14:22:32] "GET /projects HTTP/1.1" 200 -
127.0.0.1 - - [02/Sep/2021 14:22:32] "GET /jiras?num_results=15&project=all HTTP/1.1" 200 -
127.0.0.1 - - [02/Sep/2021 14:22:34] "OPTIONS /predictions HTTP/1.1" 200 -
100%|██████████| 1/1 [00:00<00:00,  2.84it/s]
127.0.0.1 - - [02/Sep/2021 14:22:39] "POST /predictions HTTP/1.1" 200 -
127.0.0.1 - - [02/Sep/2021 14:22:43] "OPTIONS /predictions HTTP/1.1" 200 -
100%|██████████| 1/1 [00:00<00:00,  4.89it/s]
127.0.0.1 - - [02/Sep/2021 14:22:47] "POST /predictions HTTP/1.1" 200 -
127.0.0.1 - - [02/Sep/2021 14:59:02] "GET /projects HTTP/1.1" 200 -
127.0.0.1 - - [02/Sep/2021 14:59:02] "GET /jira